# Advanced ML Training with RusTorch WASM
# RusTorch WASM 高度なML学習

This notebook demonstrates advanced machine learning training capabilities using RusTorch WASM, including optimizers, autograd, and neural network training.

このノートブックでは、オプティマイザ、自動微分、ニューラルネットワーク学習を含む、RusTorch WASMを使用した高度な機械学習学習機能を実演します。

In [ ]:
// Load RusTorch Jupyter integration
const script = document.createElement('script');
script.src = '../rustorch_jupyter.js';
document.head.appendChild(script);
await new Promise(resolve => { script.onload = resolve; });

// Initialize
const rustorch = new RusTorchJupyter();
await rustorch.initialize();
console.log('🚀 Advanced ML training environment ready');

## Dataset Generation / データセット生成

Generate synthetic datasets for training.

In [ ]:
// Generate synthetic classification dataset
const dist = await rustorch.distributions();
const utils = await rustorch.utils();

// Generate features (2D Gaussian clusters)
const n_samples = 1000;
const n_features = 2;

// Class 0: centered at (-1, -1)
const class0_x = dist.normal(n_samples/2, -1.0, 0.5);
const class0_y = dist.normal(n_samples/2, -1.0, 0.5);

// Class 1: centered at (1, 1)
const class1_x = dist.normal(n_samples/2, 1.0, 0.5);
const class1_y = dist.normal(n_samples/2, 1.0, 0.5);

// Combine features
const features = [...class0_x, ...class1_x, ...class0_y, ...class1_y];
const labels = [...new Array(n_samples/2).fill(0), ...new Array(n_samples/2).fill(1)];

console.log('📊 Generated synthetic dataset:');
console.log(`Features shape: [${n_samples}, ${n_features}]`);
console.log(`Labels: ${n_samples/2} samples per class`);
console.log('Sample features:', features.slice(0, 10));
console.log('Sample labels:', labels.slice(0, 10));

## Neural Network Architecture / ニューラルネットワークアーキテクチャ

Define a simple binary classifier.

In [ ]:
// Create neural network
const nn = await rustorch.neuralNetwork();

// Simple 2-layer MLP for binary classification
const layer1 = nn.createLinear(2, 8, true);   // 2 features → 8 hidden
const layer2 = nn.createLinear(8, 1, true);   // 8 hidden → 1 output

console.log('🧠 Neural network architecture:');
console.log('Layer 1: Linear(2 → 8) with bias');
console.log('Layer 2: Linear(8 → 1) with bias');
console.log('Activation: ReLU (built-in)');
console.log('Output: Sigmoid for binary classification');

## Training Loop / 学習ループ

Implement a simple training loop with SGD optimizer.

In [ ]:
// Initialize optimizer
const optimizers = await rustorch.optimizers();
const sgd = optimizers.sgd(0.01, 0.9); // learning_rate=0.01, momentum=0.9

// Training parameters
const epochs = 10;
const batch_size = 32;
const n_batches = Math.floor(n_samples / batch_size);

console.log('🎯 Training configuration:');
console.log(`Epochs: ${epochs}`);
console.log(`Batch size: ${batch_size}`);
console.log(`Batches per epoch: ${n_batches}`);
console.log('Optimizer: SGD with momentum');

In [ ]:
// Simple training simulation (pseudo-code for demonstration)
// Note: Full backpropagation requires the simplified autograd system

console.log('🔄 Training simulation:');

for (let epoch = 0; epoch < epochs; epoch++) {
    let total_loss = 0;
    
    for (let batch = 0; batch < Math.min(n_batches, 5); batch++) { // Limit for demo
        // Get batch data
        const start_idx = batch * batch_size;
        const end_idx = Math.min(start_idx + batch_size, n_samples);
        const batch_size_actual = end_idx - start_idx;
        
        // Extract batch features
        const batch_features = [];
        for (let i = start_idx; i < end_idx; i++) {
            batch_features.push(features[i], features[i + n_samples]); // x, y coordinates
        }
        
        // Forward pass
        const hidden_output = layer1.forward(batch_features, batch_size_actual, 2);
        const predictions = layer2.forward(hidden_output, batch_size_actual, 8);
        
        // Compute simple loss (mean squared error for demo)
        let batch_loss = 0;
        for (let i = 0; i < batch_size_actual; i++) {
            const target = labels[start_idx + i];
            const pred = 1 / (1 + Math.exp(-predictions[i])); // sigmoid
            batch_loss += (pred - target) ** 2;
        }
        batch_loss /= batch_size_actual;
        total_loss += batch_loss;
        
        // Simulate optimizer step
        // sgd.step(params, grads); // Would use actual gradients here
    }
    
    const avg_loss = total_loss / Math.min(n_batches, 5);
    console.log(`Epoch ${epoch + 1}/${epochs}: Loss = ${avg_loss.toFixed(4)}`);
}

console.log('✅ Training simulation completed');

## Performance Analysis / パフォーマンス分析

Analyze the performance characteristics in Jupyter environment.

In [ ]:
// Comprehensive performance analysis
console.log('🔍 Performance Analysis in Jupyter Lab:');

// Test different tensor sizes
const sizes = [100, 1000, 10000];
const results = [];

for (const size of sizes) {
    const test_data = new Array(size).fill(0).map(() => Math.random());
    const test_tensor = await rustorch.createTensor(test_data, [Math.sqrt(size), Math.sqrt(size)]);
    
    const add_time = await utils.benchmark(async () => {
        test_tensor.add(test_tensor);
    }, 50);
    
    const sum_time = await utils.benchmark(async () => {
        test_tensor.sum();
    }, 50);
    
    results.push({ size, add_time, sum_time });
    console.log(`Size ${size}: Add=${add_time.toFixed(2)}ms, Sum=${sum_time.toFixed(2)}ms`);
}

console.log('\n📊 Performance Summary:');
console.log('- Small tensors (100): Fast operations suitable for interactive development');
console.log('- Medium tensors (1000): Good performance for notebook experiments');
console.log('- Large tensors (10000): Consider WebGPU acceleration for better performance');

## Cleanup / クリーンアップ

Properly cleanup resources when done.

In [ ]:
// Cleanup (if needed for memory management)
console.log('🧹 Cleaning up resources...');

// Free tensors if memory management is critical
// Note: WASM garbage collection usually handles this automatically

console.log('✅ Jupyter Lab session cleanup completed');
console.log('💡 Tip: Use "Kernel → Restart" to fully reset the WASM environment');